1. 平移不变性
2. 局部性

对全连接层使用平移不变性和局部性得到卷积层

1. 卷积层将输入 和 核矩阵进行交叉相关，加上偏移后得到输出
2. 核矩阵 和 偏移 是可学习的参数
3. 核矩阵的大小是超参数

In [18]:
import torch
from torch import nn
from d2l import torch as d2l

In [20]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return Y


In [21]:
a = torch.tensor([[3,2],[1,2]])
b = torch.tensor([[1,2],[4,2]])

(a*b).sum(),(a*b)

(tensor(15),
 tensor([[3, 4],
         [4, 4]]))

In [22]:
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.tensor([[0, 1], [2, 3]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

卷积层

In [25]:
# 这种写法没有考虑，输入输出通道
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

一个训练的例子

In [26]:
X = torch.ones((6,8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [29]:
# 构造一个卷积核，目的是识别边界
K = torch.tensor([[1.0, -1.0]])
K

tensor([[ 1., -1.]])

In [30]:
Y = corr2d(X, K)
Y 

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [32]:
# 现在构造一个例子，学习卷积核

# 构造一个二维卷积层，它具有一个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1,2), bias=False)

X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2


for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i+1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 3.595
epoch 4, loss 0.632
epoch 6, loss 0.118
epoch 8, loss 0.025
epoch 10, loss 0.006


In [33]:
conv2d.weight.data

tensor([[[[ 0.9963, -0.9843]]]])